### BERT Pre-train on GPU

Succesfully run on Google Colab GPU as of 6/5/2020 at 1:30pm    \
Saved as model_pt11.pth      \
7 Epochs, Batch Size 32       


In [ ]:
import BERT_Project_Package as bpp

#For all your pickle needs
import pickle

#General stuff
import pandas as pd
import numpy as np

#For Train and Test Split
import random

#For Running BERT
from transformers import BertTokenizer, BertForPreTraining, BertModel, TFBertForMaskedLM, BertConfig, BertForMaskedLM
import torch

#Try 2 pre-training
import tokenizers
from transformers import Trainer, TrainingArguments
from transformers import BertTokenizer, LineByLineTextDataset, BertModel, BertConfig, BertForMaskedLM, DataCollatorForLanguageModeling

#For pre-training not by hand for BERT
from transformers import TrainingArguments
from transformers import Trainer

#Initialize optimizer for pre-training
from transformers import AdamW

#datasets
import datasets
from datasets import load_metric

In [ ]:
import json
with open('easier.json') as d:
    x_dict = json.load(d)

In [ ]:
train, validate, test = bpp.trainvaltest(x_dict)

In [ ]:
lines = train["text"]
with open('my_dat.txt', 'w') as f:
    for line in lines:
        f.write(line)
        f.write('\n')

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
dataset= LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path = './my_dat.txt',
    block_size = 128
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# Defining configuration of BERT for pretraining
config = BertConfig(
    vocab_size=50000,
    hidden_size=768, 
    num_hidden_layers=6, 
    num_attention_heads=12,
    max_position_embeddings=512
)
 
model = BertForMaskedLM(config)
print('No of parameters: ', model.num_parameters())


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# Defining training configuration\
training_args = TrainingArguments(
    output_dir='./Project',
    overwrite_output_dir=True,
    num_train_epochs=7,
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# Perfrom pre-training and save the model
trainer.train()
trainer.save_model('./Project')

In [ ]:
torch.save(model, 'model_pt11.pth')